In [ ]:
#한글분석을 위해 Konlpy설치
!pip install Konlpy
import konlpy
konlpy.__version__
#0.5.2

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 92kB 14.7MB/s 
     |████████████████████████████████| 460kB 47.5MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


'0.5.2'

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
#파일업로드
from google.colab import files

uploaded = files.upload()
filename = list(uploaded.keys())[0] 

Saving ratings_train.txt to ratings_train.txt


In [ ]:
uploaded = files.upload()
testfile = list(uploaded.keys())[0] 

Saving ratings_test.txt to ratings_test.txt


## 데이터전처리

In [ ]:
import pandas as pd
train_df= pd.read_table(filename)
test_df= pd.read_table(testfile)

In [ ]:
train_df.shape

(150000, 3)

In [ ]:
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [ ]:
#트레인 데이터 셋에 적용
train_df.drop_duplicates(subset=['document'], inplace=True)  #중복제거
train_df = train_df.dropna(how='any') #널값제거
train_df.shape

(146182, 3)

In [ ]:
#긍정1 부정0 레이블값의 갯수
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

In [ ]:
#테스트 데이터 셋에 적용
test_df.drop_duplicates(subset=['document'], inplace=True)  # 중복제거
test_df = test_df.dropna(how='any') #널값제거
test_df.shape

(49157, 3)

## 한글텍스트전처리

In [ ]:
#트레인 데이터 셋에 적용
train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_df['document'].replace('', np.nan, inplace=True) #빈데이터
train_df = train_df.dropna(how = 'any') 
train_df.shape

(145791, 3)

In [ ]:
#테스트 데이터 셋에 적용

In [ ]:
test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_df['document'].replace('', np.nan, inplace=True) #빈데이터
test_df = test_df.dropna(how = 'any') # Null값제거
test_df.shape

(48995, 3)

## 한글형태소분석

In [ ]:
from konlpy.tag import Okt 
# 토큰화와 불용어 제거
import tqdm.notebook as tn
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
okt = Okt()

X_train=[]
for sentence in tn.tqdm(train_df['document']):
    temp_X = []
    temp_X = okt.morphs(sentence, stem=True) 
    temp_X = [word for word in temp_X if not word in stopwords] 
    X_train.append(temp_X) 

In [ ]:
X_test = []
for sentence in tn.tqdm(test_df['document']):
    morphs = okt.morphs(sentence, stem=True) 
    temp_X = ' '.join([word for word in morphs if not word in stopwords]) 
    X_test.append(temp_X)

In [ ]:
X_train[:3]

[['아', '더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다']]

## 케라스인코딩

In [ ]:
# 케라스인코딩하기위한 정수인코딩
max_words = 35000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train[:3] 

[[50, 454, 16, 260, 659],
 [933, 457, 41, 602, 1, 214, 1449, 24, 961, 675, 19],
 [386, 2444, 25015, 2315, 5671, 2, 222, 9]]

## 리뷰길이 맞추기

In [ ]:
print('리뷰의 최대 길이 :', max(len(s) for s in X_train))
print('리뷰의 평균 길이 :', sum(map(len, X_train))/len(X_train))

리뷰의 최대 길이 : 69
리뷰의 평균 길이 : 10.911133060339802


In [ ]:
max_len=30
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
y_train = train_df['label'].values
y_test = test_df['label'].values 

## 1)LSTM 모델정의/설정/학습0.8479

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# 모델 구성하기
model = Sequential([
                    Embedding(max_words, 100),
                    LSTM(128),
                    Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 100)         3500000   
_________________________________________________________________
lstm_20 (LSTM)               (None, 128)               117248    
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 129       
Total params: 3,617,377
Trainable params: 3,617,377
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 모델 학습과정 설정하기
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
checkpointer = ModelCheckpoint(filepath='naverbest-model.h5', monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
history = model.fit(X_train, y_train, epochs=200, batch_size=50,
                    validation_split=0.2, verbose=1, callbacks=[checkpointer, early_stopping])

Epoch 1/200
2333/2333 [==============================] - 78s 33ms/step - loss: 0.4338 - accuracy: 0.7935 - val_loss: 0.3459 - val_accuracy: 0.8479

Epoch 00001: val_loss improved from inf to 0.34589, saving model to naverbest-model.h5
Epoch 2/200
2333/2333 [==============================] - 76s 33ms/step - loss: 0.2786 - accuracy: 0.8828 - val_loss: 0.3416 - val_accuracy: 0.8534

Epoch 00002: val_loss improved from 0.34589 to 0.34160, saving model to naverbest-model.h5
Epoch 3/200
2333/2333 [==============================] - 75s 32ms/step - loss: 0.2202 - accuracy: 0.9111 - val_loss: 0.3709 - val_accuracy: 0.8491

Epoch 00003: val_loss did not improve from 0.34160
Epoch 4/200
2333/2333 [==============================] - 74s 32ms/step - loss: 0.1716 - accuracy: 0.9314 - val_loss: 0.4223 - val_accuracy: 0.8448

Epoch 00004: val_loss did not improve from 0.34160
Epoch 5/200
2333/2333 [==============================] - 73s 31ms/step - loss: 0.1309 - accuracy: 0.9500 - val_loss: 0.4864 - va

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model('naverbest-model.h5')

In [ ]:
# 테스트 정확도
acc = best_model.evaluate(X_test, y_test, verbose=2)[1]
print(f"Accuracy:{acc:.4f}")
#모델은 그대로, epochs=200늘리고, batch_size=50줄여서
#기존0.8459보다 조금 높은 0.8479나왔다!!;;

1532/1532 - 4s - loss: 0.3501 - accuracy: 0.8479
Accuracy:0.8479


## 2)LSTM+CNN 모델정의/설정/학습/평가0.7754

In [ ]:
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D
# Conv1D 와 LSTM 모델
model3 = Sequential([
                    Embedding(max_words, 50), #줄이고
                    Dropout(0.5),
                    Conv1D(64, 5, padding='valid', activation='relu'),
                    MaxPooling1D(pool_size=4),
                    LSTM(300),#늘리고
                    Dense(1, activation='sigmoid')
])
model3.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, None, 50)          1750000   
_________________________________________________________________
dropout_7 (Dropout)          (None, None, 50)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 64)          16064     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
lstm_29 (LSTM)               (None, 300)               438000    
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 301       
Total params: 2,204,365
Trainable params: 2,204,365
Non-trainable params: 0
___________________________________________

In [ ]:
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='naverbest-model3.h5', monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history3 = model3.fit(X_train, y_train, epochs=200, batch_size=60,
                    validation_split=0.2, verbose=1, callbacks=[checkpointer,early_stopping])

Epoch 1/200
1944/1944 [==============================] - 38s 18ms/step - loss: 0.5452 - accuracy: 0.6930 - val_loss: 0.4478 - val_accuracy: 0.7765

Epoch 00001: val_loss improved from inf to 0.44781, saving model to naverbest-model3.h5
Epoch 2/200
1944/1944 [==============================] - 36s 18ms/step - loss: 0.4135 - accuracy: 0.8007 - val_loss: 0.4431 - val_accuracy: 0.7810

Epoch 00002: val_loss improved from 0.44781 to 0.44307, saving model to naverbest-model3.h5
Epoch 3/200
1944/1944 [==============================] - 36s 19ms/step - loss: 0.3734 - accuracy: 0.8199 - val_loss: 0.4545 - val_accuracy: 0.7783

Epoch 00003: val_loss did not improve from 0.44307
Epoch 4/200
1944/1944 [==============================] - 36s 18ms/step - loss: 0.3424 - accuracy: 0.8360 - val_loss: 0.4761 - val_accuracy: 0.7776

Epoch 00004: val_loss did not improve from 0.44307
Epoch 5/200
1944/1944 [==============================] - 36s 18ms/step - loss: 0.3192 - accuracy: 0.8471 - val_loss: 0.4919 - 

In [ ]:
best_model = load_model('naverbest-model3.h5')
acc = best_model.evaluate(X_test, y_test, verbose=2)[1]
print(f"Accuracy:{acc:.4f}")
# Embedding(max_words, 50)부분을 100에서 50으로줄이고, LSTM(300)부분을 60에서 300으로 늘렸음, epochs=200 늘림
#기존 0.7725보다 높으면 되고 위에 0.8479보다 높으면 좋음;
#0.7754

1532/1532 - 3s - loss: 0.4516 - accuracy: 0.7754
Accuracy:0.7754


## 3)model2 : 0.8462

In [ ]:
model2 = Sequential()
model2.add(Embedding(max_words, 50, input_length=max_len))
model2.add(LSTM(64))
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(1, activation='sigmoid'))

model2.summary()

Model: "sequential_44"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 30, 50)            1750000   
_________________________________________________________________
lstm_46 (LSTM)               (None, 64)                29440     
_________________________________________________________________
dense_28 (Dense)             (None, 256)               16640     
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 1)                 257       
Total params: 1,796,337
Trainable params: 1,796,337
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
checkpointer = ModelCheckpoint(filepath='naverbest-model2.h5', monitor='val_loss', 
                               verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
history2 = model2.fit(X_train, y_train, epochs=200, batch_size=60,
                    validation_split=0.2, verbose=1, callbacks=[checkpointer,early_stopping])

Epoch 1/200
1944/1944 [==============================] - 39s 19ms/step - loss: 0.4418 - accuracy: 0.7881 - val_loss: 0.3563 - val_accuracy: 0.8429

Epoch 00001: val_loss improved from inf to 0.35635, saving model to naverbest-model2.h5
Epoch 2/200
1944/1944 [==============================] - 37s 19ms/step - loss: 0.2891 - accuracy: 0.8771 - val_loss: 0.3458 - val_accuracy: 0.8500

Epoch 00002: val_loss improved from 0.35635 to 0.34584, saving model to naverbest-model2.h5
Epoch 3/200
1944/1944 [==============================] - 36s 19ms/step - loss: 0.2305 - accuracy: 0.9056 - val_loss: 0.3809 - val_accuracy: 0.8485

Epoch 00003: val_loss did not improve from 0.34584
Epoch 4/200
1944/1944 [==============================] - 36s 19ms/step - loss: 0.1851 - accuracy: 0.9259 - val_loss: 0.4260 - val_accuracy: 0.8438

Epoch 00004: val_loss did not improve from 0.34584
Epoch 5/200
1944/1944 [==============================] - 35s 18ms/step - loss: 0.1488 - accuracy: 0.9419 - val_loss: 0.4842 - 

In [ ]:
best_model = load_model('naverbest-model2.h5')
acc = best_model.evaluate(X_test, y_test, verbose=2)[1]
print(f"Accuracy:{acc:.4f}")
#0.8462

1532/1532 - 3s - loss: 0.3531 - accuracy: 0.8462
Accuracy:0.8462
